<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

This notebook is part of the capstone project in the IBM Applied Data Science Specialization on Coursera. It uses data from Wikipedia, the Foursquare API to explore common venue categories in each postal code, and the k-means clustering algorithm to cluster neighbourhoods in Toronto, based on this feature. Finally, Folium is used to visualize the neighbourhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Firstly, let's import the packages we will need...

In [92]:
# standard libraries for data manipulation
import pandas as pd
import numpy as np

import requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# use geocoder to get the long, lat coordinates for each postal code
import geocoder

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# map visualisation
import folium

print('Libraries imported!')

Libraries imported!


## 1. Download and explore dataset

In [4]:
# wiki page with table we want to read
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url, header=0)[0]

# remove irrelevant boroughs
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) 

# group all neighborhoods in the same postal code, comma separated
df['Neighborhood'] = df.groupby('Postal code')['Neighborhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()

In [5]:
# set neighborhood to borough if neighborhood is missing
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    else:
        pass
    row['Neighborhood'] = row['Neighborhood'].replace(' /', ',')

In [6]:
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
rows = df.shape[0]
boroughs = len(df['Borough'].unique())

print(f'The dataset has {rows} rows, and {boroughs} unique boroughs.')

The dataset has 103 rows, and 10 unique boroughs.


### Use geocoder to get the lat, long coordinates

This method runs too slow, see csv method below...
```python
postal_codes = df['Postal code'].unique()
postal_code = 'M3A'

# initialize variable to None
lat_lng_coords = None

# loop until we get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

In [8]:
# csv with long, lat data for each postal code
url = 'http://cocl.us/Geospatial_data'

df_ = pd.read_csv(url)
df = df.merge(df_, left_on='Postal code', right_on='Postal Code')
columns=['Postal code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
df = df[columns]

In [9]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [10]:
# Toronto lat, long values
lat = 43.6532
long = -79.3832

# create map of New Toronto using latitude and longitude values
map_ = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
    
map_

In [11]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [12]:
# lets use postal code M5A as an example first...
M5A = toronto_data[toronto_data['Postal code'] == "M5A"]
M5A_lat = M5A.at[0,'Latitude']
M5A_long = M5A.at[0,'Longitude']

print(f'The latitude and longitude for M5A are {M5A_lat} and {M5A_long}, respectively.')

The latitude and longitude for M5A are 43.6542599 and -79.3606359, respectively.


In [13]:
id_ = '3ABQ3SB5JHDSEX0OIK1NUG4IU12GDGW2R3JFOAZL2N3NRPHS'
secret = 'FPST5NAR1GEG0FO2BRJFCS1ORB5TWCIRIUBDQL5W3XDHDJDW'
version = '20200101'
rad = 500
lim = 100

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={id_}&client_secret={secret}&v={version}&ll={M5A_lat},{M5A_long}&radius={rad}&limit={lim}'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eae741202a172002467c85a'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

We can see from the results that all the information is held in the 'items' key. Let's use a function to extract the useful information we want...

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


How many venues do we have for the M5A postcode?

In [17]:
print(f'Foursquare returned {nearby_venues.shape[0]} venues.')

Foursquare returned 48 venues.


## 2. Explore all Neighborhoods in Toronto

#### Create a function that repeats the process above for all neighborhoods in Toronto....

In [69]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id= \
            {id_}&client_secret={secret}&v={version}&ll={lat},{long}&radius={rad}&limit={lim}'    
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    nearby_venues = nearby_venues[nearby_venues['Venue Category'] != 'Neighborhood']

    return(nearby_venues)

In [70]:
toronto_venues = get_nearby_venues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [72]:
print(toronto_venues.shape)
toronto_venues.head()

(2381, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"Regent Park, Harbourfront",43.65426,-79.360636,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Regent Park, Harbourfront",43.65426,-79.360636,Indigo,43.653515,-79.380696,Bookstore
3,"Regent Park, Harbourfront",43.65426,-79.360636,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
4,"Regent Park, Harbourfront",43.65426,-79.360636,CF Toronto Eaton Centre,43.654540,-79.380677,Shopping Mall
5,"Regent Park, Harbourfront",43.65426,-79.360636,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot


How many venues per neighborhood?

In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,68,68,68,68,68,68
"Brockton, Parkdale Village, Exhibition Place",53,53,53,53,53,53
Business reply mail Processing CentrE,76,76,76,76,76,76
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",3,3,3,3,3,3
Central Bay Street,91,91,91,91,91,91
Christie,70,70,70,70,70,70
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,20,20,20,20,20,20


#### How many unique categories are there across all neighborhoods?

In [79]:
print("There are {} unique venue categories.".format(len(toronto_venues['Venue Category'].unique())))

There are 163 unique venue categories.


# 3. Analyse each neighborhood

In [80]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
toronto_onehot.shape

(2381, 164)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [82]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.029412,0.014706,0.000000,0.000000,0.000000,0.00,0.00,...,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.018868,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.013158,0.000000,0.000000,0.000000,0.013158,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.013158
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.010989,0.010989,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.010989,0.000000,0.010989,0.000000,0.000000,0.00,0.000000,0.010989
5,Christie,0.014286,0.014286,0.000000,0.014286,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,0.014286,0.00,0.000000,0.014286
6,Church and Wellesley,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.025641
7,"Commerce Court, Victoria Hotel",0.000000,0.040000,0.000000,0.010000,0.000000,0.000000,0.030000,0.00,0.00,...,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000
8,Davisville,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [84]:
num_top_venues = 5

for nbh in toronto_grouped['Neighborhood']:
    print("-----"+nbh+"-----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == nbh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

-----Berczy Park-----
                 venue  freq
0          Coffee Shop  0.13
1                Hotel  0.06
2        Deli / Bodega  0.04
3                 Café  0.04
4  Japanese Restaurant  0.04


-----Brockton, Parkdale Village, Exhibition Place-----
           venue  freq
0  Boat or Ferry  0.11
1    Coffee Shop  0.09
2    Pizza Place  0.06
3           Café  0.04
4    Music Venue  0.04


-----Business reply mail Processing CentrE-----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.07
2     Ramen Restaurant  0.04
3                Diner  0.04
4     Sushi Restaurant  0.04


-----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport-----
             venue  freq
0    Boat or Ferry  0.33
1    Historic Site  0.33
2  Harbor / Marina  0.33
3   Adult Boutique  0.00
4       Playground  0.00


-----Central Bay Street-----
                       venue  freq
0                Coffee Shop  0.15
1             S

#### Let's make that into a dataframe...

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Deli / Bodega,Japanese Restaurant,Café,Pub,Aquarium,Salad Place,Restaurant,Chinese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Boat or Ferry,Coffee Shop,Pizza Place,Brewery,Café,Fried Chicken Joint,Restaurant,Music Venue,Park,Convenience Store
2,Business reply mail Processing CentrE,Coffee Shop,Japanese Restaurant,Diner,Ramen Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Breakfast Spot,Park,Dance Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Historic Site,Boat or Ferry,Harbor / Marina,Yoga Studio,Dog Run,Food & Drink Shop,Flower Shop,Fast Food Restaurant,Falafel Restaurant,Event Space
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Bar,Café,Ramen Restaurant,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Restaurant


# 4. Cluster Neighborhoods

Run k-means to cluster neighborhoods into 5 clusters...

In [88]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 0, 0, 0, 0, 0, 3])

In [89]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Clothing Store,Coffee Shop,Restaurant,Hotel,Plaza,Breakfast Spot,Middle Eastern Restaurant,Seafood Restaurant,Cosmetics Shop,Bubble Tea Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Japanese Restaurant,Diner,Gay Bar,Sandwich Place,Ramen Restaurant,Pizza Place,Sushi Restaurant,Burger Joint,Gastropub
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Sandwich Place,Clothing Store,Hotel,Bar,Diner,Café,Cosmetics Shop,Fried Chicken Joint,Movie Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Gym,Thai Restaurant,Bookstore,Deli / Bodega,Steakhouse
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Yoga Studio,Building,Salon / Barbershop,Park,Dog Run,Flower Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [93]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters